#### From https://github.com/chenyuntc/pytorch-book.git

In [1]:
from __future__ import print_function
import torch  as t
t.__version__

'1.5.0'

從接口的角度來講，對tensor的操作可分為兩類：

1. `torch.function`，如`torch.save`等。
2. 另一類是`tensor.function`，如`tensor.view`等。

為方便使用，對tensor的大部分操作同時支持這兩類接口，在本書中不做具體區分，如`torch.sum (torch.sum(a, b))`與`tensor.sum (a. sum(b))`功能等價。

而從存儲的角度來講，對tensor的操作又可分為兩類：

1. 不會修改自身的數據，如 `a.add(b)`， 加法的結果會返回一個新的tensor。
2. 會修改自身的數據，如 `a.add_(b)`， 加法的結果仍存儲在a中，a被修改了。

函數名以`_`結尾的都是inplace方式, 即會修改調用者自己的數據，在實際應用中需加以區分。

#### 創建Tensor

在PyTorch中新建tensor的方法有很多，具體如表3-1所示。

表3-1: 常見新建tensor的方法

|函數|功能|
|:---:|:---:|
|Tensor(\*sizes)|基礎構造函數|
|tensor(data,)|類似np.array的構造函數|
|ones(\*sizes)|全1Tensor|
|zeros(\*sizes)|全0Tensor|
|eye(\*sizes)|對角線為1，其他為0|
|arange(s,e,step|從s到e，步長為step|
|linspace(s,e,steps)|從s到e，均勻切分成steps份|
|rand/randn(\*sizes)|均勻/標準分佈|
|normal(mean,std)/uniform(from,to)|正態分佈/均勻分佈|
|randperm(m)|隨機排列|

這些創建方法都可以在創建的時候指定數據類型dtype和存放device(cpu/gpu).


其中使用`Tensor`函數新建tensor是最複雜多變的方式，它既可以接收一個list，並根據list的數據新建tensor，也能根據指定的形狀新建tensor，還能傳入其他的tensor，下面舉幾個例子。

In [2]:
a = t.Tensor(2, 3)
a

tensor([[7.3867e+20, 9.2358e-01, 1.8061e+28],
        [4.4378e+27, 5.5388e-14, 1.6109e-19]])

In [3]:
# Use List
b = t.Tensor([[1, 2, 3], [4, 5, 6]])
b

tensor([[1., 2., 3.],
        [4., 5., 6.]])

In [4]:
b.tolist()

[[1.0, 2.0, 3.0], [4.0, 5.0, 6.0]]

In [5]:
print(b.size()) # b.shape
print(b.numel())

torch.Size([2, 3])
6


In [6]:
c = t.Tensor(b.size())
c

tensor([[1.8370e+25, 1.4603e-19, 1.2102e+25],
        [1.6217e-19, 3.0881e+29, 1.2102e+25]])

In [7]:
t.ones(2,3)

tensor([[1., 1., 1.],
        [1., 1., 1.]])

In [8]:
t.zeros(2,3)

tensor([[0., 0., 0.],
        [0., 0., 0.]])

In [9]:
t.arange(1,6,2)

tensor([1, 3, 5])

In [10]:
t.linspace(1, 10, 3)

tensor([ 1.0000,  5.5000, 10.0000])

In [11]:
t.rand(2, 3, device=t.device('cpu'))

tensor([[0.9650, 0.9326, 0.0419],
        [0.1856, 0.9682, 0.4638]])

In [12]:
t.randperm(6)

tensor([0, 4, 1, 2, 5, 3])

In [13]:
t.eye(3, 3, dtype=t.int)

tensor([[1, 0, 0],
        [0, 1, 0],
        [0, 0, 1]], dtype=torch.int32)

In [14]:
scalar = t.tensor(3.14159) 
print('scalar: %s, shape of sclar: %s' %(scalar, scalar.shape))

scalar: tensor(3.1416), shape of sclar: torch.Size([])


In [15]:
vector = t.tensor([5,3])
print('vector: %s, shape of vector: %s' %(vector, vector.shape))

vector: tensor([5, 3]), shape of vector: torch.Size([2])


In [16]:
matrix = t.tensor([[0.1, 1.2], [2.2, 3.1], [4.9, 5.2]])
matrix,matrix.shape

(tensor([[0.1000, 1.2000],
         [2.2000, 3.1000],
         [4.9000, 5.2000]]),
 torch.Size([3, 2]))

In [17]:
t.tensor([[0.11111, 0.222222, 0.3333333]], dtype=t.float64, device=t.device('cpu'))

tensor([[0.1111, 0.2222, 0.3333]], dtype=torch.float64)

In [18]:
empty_tensor = t.tensor([])
empty_tensor.shape

torch.Size([0])

## Some Useful Functions

通過`tensor.view`方法可以調整tensor的形狀，但必須保證調整前後元素總數一致。 `view`不會修改自身的數據，返回的新tensor與源tensor共享內存，也即更改其中的一個，另外一個也會跟著改變。在實際應用中可能經常需要添加或減少某一維度，這時候`squeeze`和`unsqueeze`兩個函數就派上用場了。

In [19]:
a = t.arange(0, 6)
a.view(2, 3)

tensor([[0, 1, 2],
        [3, 4, 5]])

In [20]:
b = a.view(-1, 3) # -1 => it will calculate the dimension itself
b.shape

torch.Size([2, 3])

In [21]:
b.unsqueeze(1)
print(b[:, None].shape)
print(b)

torch.Size([2, 1, 3])
tensor([[0, 1, 2],
        [3, 4, 5]])


In [22]:
b.unsqueeze(-2) # -2表示倒數第二個維度

tensor([[[0, 1, 2]],

        [[3, 4, 5]]])

In [23]:
c = b.view(1, 1, 1, 2, 3)
print(c)
print(c.squeeze(0)) # 壓縮第0維的 "1"
print(c.squeeze())

tensor([[[[[0, 1, 2],
           [3, 4, 5]]]]])
tensor([[[[0, 1, 2],
          [3, 4, 5]]]])
tensor([[0, 1, 2],
        [3, 4, 5]])


In [24]:
a = t.randn(3, 4)
a

tensor([[-0.0313, -0.2236,  0.6810, -1.1241],
        [-0.9615,  2.2381, -0.9188,  0.7759],
        [-1.2850,  0.1560, -0.1778,  1.6034]])

In [25]:
a[0]

tensor([-0.0313, -0.2236,  0.6810, -1.1241])

In [26]:
a[0, :]

tensor([-0.0313, -0.2236,  0.6810, -1.1241])

In [27]:
a[:, 0]

tensor([-0.0313, -0.9615, -1.2850])

In [28]:
print(a[0][2])
print(a[0][-1])

tensor(0.6810)
tensor(-1.1241)


In [29]:
# None類似於np.newaxis, 為a新增了一個軸
# 等價於a.view(1, a.shape[0], a.shape[1])
a[None].shape

torch.Size([1, 3, 4])

In [32]:
print(a[:, None, :].shape)
print(a[:,None,:,None,None].shape)

torch.Size([3, 1, 4])
torch.Size([3, 1, 4, 1, 1])


In [33]:
a

tensor([[-0.0313, -0.2236,  0.6810, -1.1241],
        [-0.9615,  2.2381, -0.9188,  0.7759],
        [-1.2850,  0.1560, -0.1778,  1.6034]])

其它常用的選擇函數如表3-2所示。


函數|功能|
:---:|:---:|
index_select(input, dim, index)|在指定維度dim上選取，比如選取某些行、某些列
masked_select(input, mask)|例子如上，a[a>0]，使用ByteTensor進行選取
non_zero(input)|非0元素的下標
gather(input, dim, index)|根據index，在dim維度上選取數據，輸出的size與index一樣


`gather`是一個比較複雜的操作，對一個2維tensor，輸出的每個元素如下：

```python
out[i][j] = input[index[i][j]][j] # dim=0
out[i][j] = input[i][index[i][j]] # dim=1
```
三維tensor的`gather`操作同理，下面舉幾個例子。

In [36]:
a = t.arange(0, 16).view(4,4)
a

tensor([[ 0,  1,  2,  3],
        [ 4,  5,  6,  7],
        [ 8,  9, 10, 11],
        [12, 13, 14, 15]])

In [38]:
index = t.LongTensor([[0,1,2,3]])
a.gather(0, index)

tensor([[ 0,  5, 10, 15]])

In [43]:
index = t.LongTensor([[3,2,1,0]]).t()
print(index)
print(a.gather(1, index))

tensor([[3],
        [2],
        [1],
        [0]])
tensor([[ 3],
        [ 6],
        [ 9],
        [12]])


#### 逐元素操作

這部分操作會對tensor的每一個元素(point-wise，又名element-wise)進行操作，此類操作的輸入與輸出形狀一致。常用的操作如表3-4所示。

表3-4: 常見的逐元素操作

|函數|功能|
|:--:|:--:|
|abs/sqrt/div/exp/fmod/log/pow..|絕對值/平方根/除法/指數/求餘/求冪..|
|cos/sin/asin/atan2/cosh..|相關三角函數|
|ceil/round/floor/trunc| 上取整/四捨五入/下取整/只保留整數部分|
|clamp(input, min, max)|超過min和max部分截斷|
|sigmod/tanh..|激活函數

對於很多操作，例如div、mul、pow、fmod等，PyTorch都實現了運算符重載，所以可以直接使用運算符。如`a ** 2` 等價於`torch.pow(a,2)`, `a * 2`等價於`torch.mul(a,2)`。

其中`clamp(x, min, max)`的輸出滿足以下公式：
$$
y_i =
\begin{cases}
min, & \text{if } x_i \lt min \\
x_i, & \text{if } min \le x_i \le max \\
max, & \text{if } x_i \gt max\\
\end{cases}
$$
`clamp`常用在某些需要比較大小的地方，如取一個tensor的每個元素與另一個數的較大值。

In [45]:
a = t.arange(0, 6).view(2, 3).float()
print(a)
print(t.cos(a))

tensor([[0., 1., 2.],
        [3., 4., 5.]])
tensor([[ 1.0000,  0.5403, -0.4161],
        [-0.9900, -0.6536,  0.2837]])


In [47]:
print(a % 3)
print(a ** 2)

tensor([[0., 1., 2.],
        [0., 1., 2.]])
tensor([[ 0.,  1.,  4.],
        [ 9., 16., 25.]])


In [48]:
print(a)
t.clamp(a, min=3)

tensor([[0., 1., 2.],
        [3., 4., 5.]])


tensor([[3., 3., 3.],
        [3., 4., 5.]])

#### 歸併操作
此類操作會使輸出形狀小於輸入形狀，並可以沿著某一維度進行指定操作。如加法`sum`，既可以計算整個tensor的和，也可以計算tensor中每一行或每一列的和。常用的歸併操作如表3-5所示。

表3-5: 常用歸併操作

|函數|功能|
|:---:|:---:|
|mean/sum/median/mode|均值/和/中位數/眾數|
|norm/dist|範數/距離|
|std/var|標準差/方差|
|cumsum/cumprod|累加/累乘|

以上大多數函數都有一個參數**`dim`**，用來指定這些操作是在哪個維度上執行的。關於dim(對應於Numpy中的axis)的解釋眾說紛紜，這裡提供一個簡單的記憶方式：

假設輸入的形狀是(m, n, k)

- 如果指定dim=0，輸出的形狀就是(1, n, k)或者(n, k)
- 如果指定dim=1，輸出的形狀就是(m, 1, k)或者(m, k)
- 如果指定dim=2，輸出的形狀就是(m, n, 1)或者(m, n)

size中是否有"1"，取決於參數`keepdim`，`keepdim=True`會保留維度`1`。注意，以上只是經驗總結，並非所有函數都符合這種形狀變化方式，如`cumsum`。

In [49]:
b = t.ones(2, 3)
b.sum(dim = 0, keepdim=True)

tensor([[2., 2., 2.]])

In [50]:
# keepdim=False，不保留维度"1"，注意形状
b.sum(dim=0, keepdim=False)

tensor([2., 2., 2.])